In [1]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
from resources import db_connect
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sqlalchemy
import pytz
import numpy as np

import uuid


In [2]:
main = db_connect('main')

dtypes = {"trans_date":sqlalchemy.Date()
}

In [3]:
d = datetime.today() 
d = d - relativedelta(months=12)

In [4]:
d

datetime.datetime(2020, 6, 10, 11, 21, 12, 641040)

In [5]:
hdrs = ["trans_date","description","original_description","amount","transaction_type","category","account_name","labels","notes",'recid','start_date']

In [6]:
transactions_mapped = pd.read_sql('select * from mint.transactions_history'
                             , con=main.conn
                             , parse_dates=['trans_date']
                             )

In [7]:
transactions_mapped= transactions_mapped[(transactions_mapped['trans_date'] > d)]

In [8]:
# transactions_mapped['recid'] = transactions_mapped['recid'].astype('str') 
transactions_mapped['recid']=transactions_mapped.recid.apply(str)


In [9]:
# mapped_limit = [max(transactions_mapped['trans_date']),min(transactions_mapped['trans_date'])]
mapped_limit = max(transactions_mapped['trans_date'])

In [10]:
mapped_limit

Timestamp('2021-06-03 00:00:00')

In [11]:
transactions_unmapped = pd.read_csv('D:/Git/local_db/personal/data/mint/transactions.csv',parse_dates=['date'])
transactions_unmapped.rename(columns = {'date':'trans_date'},inplace=True)
u = datetime.utcnow()
u = u.replace(tzinfo = pytz.utc)
transactions_unmapped['start_date'] = u 



In [12]:
transactions_unmapped['recid'] = [uuid.uuid4() for _ in range(len(transactions_unmapped.index))]
transactions_unmapped['recid']=transactions_unmapped.recid.apply(str)

In [13]:
transactions_new = transactions_unmapped[(transactions_unmapped['trans_date'] > mapped_limit)]

In [14]:
transactions_new

,trans_date,description,original_description,amount,transaction_type,category,account_name,labels,notes,start_date,recid
0,2021-06-10,CHURCHK EY PUB EDMONDS WA,POS DEBIT CHURCHK,40.91,debit,Restaurants,PREMIER PLUS CKG,NaN,NaN,2021-06-10 18:22:00.084606+00:00,a8d0e299-c904-4e10-9ab1-c10b5a1df0d2
1,2021-06-10,Amazon,POS DEBIT AMZN Mk,31.95,debit,Shopping,PREMIER PLUS CKG,NaN,Earbuds,2021-06-10 18:22:00.084606+00:00,8326401d-a1b5-4ad8-8edc-098610fb5380
2,2021-06-09,Trade,TRADE COFFEE CO DRINKTRADE.CO MA,25.00,debit,Coffee Shops,PREMIER PLUS CKG,NaN,NaN,2021-06-10 18:22:00.084606+00:00,03e419f4-c003-470f-9a47-f226256cf846
3,2021-06-09,"TEKSYSTEMS, INC. DIR DEP PPD ID: 5522010575","TEKSYSTEMS, INC. DIR DEP",1069.62,credit,Paycheck,PREMIER PLUS CKG,NaN,NaN,2021-06-10 18:22:00.084606+00:00,5dcb3760-d207-43c2-8407-2292434c695d
4,2021-06-09,ONLYFANS 888-6880458 FL,ONLYFANS 888-6880458 FL,4.37,debit,Entertainment,Alaska Airlines Visa Signature,NaN,NaN,2021-06-10 18:22:00.084606+00:00,2f556152-e9c4-45ce-a505-3fc18a46eb52
5,2021-06-08,TWP*SUB2873298 WAPO.COM DC 06/07,TWP*SUB2873298 WAPO.COM DC,9.99,debit,Subscription_Service,PREMIER PLUS CKG,NaN,NaN,2021-06-10 18:22:00.084606+00:00,d5c30527-a887-492b-95a0-8c95385bff63
6,2021-06-07,BK OF AMER VISA ONLINE payment CKF9 58241099PO...,BK OF AMER VISA ONLINE PMT CKF9,27.57,debit,Credit Card Payment,PREMIER PLUS CKG,NaN,NaN,2021-06-10 18:22:00.084606+00:00,3db35441-fbfc-4cc3-8e09-70af00307652
7,2021-06-07,PERSHING BROKERAGE PPD ID: 1132741729,PERSHING BROKERAGE,4000.00,debit,Financial Advisor,PREMIER PLUS CKG,NaN,NaN,2021-06-10 18:22:00.084606+00:00,d34dc8fd-5594-47c3-a883-5d6be8b4b053
8,2021-06-07,XHALE GAL 127 N 85TH S SEATTLE W A 06/07,XHALE GAL 127 N 85TH S SEATTLE W,19.83,debit,Entertainment,PREMIER PLUS CKG,NaN,NaN,2021-06-10 18:22:00.084606+00:00,a5aaf1b7-d2aa-4967-a2bf-58bec90fa9ce
9,2021-06-07,Google,GOOGLE*GOOGLE FI SUPPORT.GOOGL C,55.87,debit,Mobile Phone,PREMIER PLUS CKG,NaN,NaN,2021-06-10 18:22:00.084606+00:00,dfdaf69d-c197-415c-9c51-0571c3bcb209


In [15]:
dtypes = {"trans_date":sqlalchemy.Date()
}

In [16]:
## Write these to mint.transactions_history
transactions_new.to_sql(name='transactions_history',
                        con=main.engine
                        ,schema='mint'
                        ,if_exists='append'
                        ,dtype=dtypes
                        ,index=False)

In [17]:
transactions_unmapped = transactions_unmapped[(transactions_unmapped['trans_date'] > d) &
                                              (transactions_unmapped['trans_date'] < mapped_limit)]

In [18]:
# transactions_unmapped.sort_values(by='trans_date',ascending=False)

In [19]:
lon = ['trans_date','description','original_description','amount','transaction_type','category','account_name','labels','notes']
ron = ['trans_date','description','original_description','amount','transaction_type','category','account_name','labels','notes']


In [20]:
matched_results = fuzzymatcher.fuzzy_left_join(
                                            df_left= transactions_mapped,
                                            df_right= transactions_unmapped,
                                            left_on = lon,
                                            right_on = ron,
                                            left_id_col='recid',
                                            right_id_col='recid')

In [21]:
matched_results.best_match_score.describe()


count    933.000000
mean       1.932954
std        0.586822
min       -1.149368
25%        1.526416
50%        1.850074
75%        2.218059
max        4.375674
Name: best_match_score, dtype: float64

In [22]:
print(len(matched_results))

933


In [23]:
matched_results = matched_results[matched_results['best_match_score']>=1.579446]

In [24]:
print(len(matched_results))

648


In [25]:
# {c:i for i ,c in enumerate(matched_results.columns)}
fields = {
'best_match_score': 0,
 '__id_left': 'recid',
 '__id_right': 2,
 'trans_date_left': 'trans_date',
 'description_left': 4,
 'original_description_left': 5,
 'amount_left': 'amount',
 'transaction_type_left': 'transaction_type',
 'category_left': 8,
 'account_name_left': 'account_name',
 'labels_left': 10,
 'notes_left': 11,
 'recid_left': 12,
 'start_date_left': 13,
 'trans_date_right': 14,
 'description_right': 'description',
 'original_description_right': 'original_description',
 'amount_right': 17,
 'transaction_type_right': 18,
 'category_right': 'category',
 'account_name_right': 20,
 'labels_right': 'labels',
 'notes_right': 'notes',
 'start_date_right': 'start_date',
 'recid_right': 24}

In [26]:
matched_results.rename(columns=fields,inplace=True)

In [27]:
matched_results = matched_results[["trans_date"
                                   ,"description"
                                   ,"original_description"
                                   ,"amount"
                                   ,"transaction_type"
                                   ,"category"
                                   ,"account_name"
                                   ,"labels"
                                   ,"notes"
                                   ,'recid'
                                   ,'start_date']]

In [28]:
matched_results.dtypes

trans_date                   datetime64[ns]
description                          object
original_description                 object
amount                              float64
transaction_type                     object
category                             object
account_name                         object
labels                               object
notes                                object
recid                                object
start_date              datetime64[ns, UTC]
dtype: object

In [29]:
matched_results.recid.value_counts()

144a0dc1-c499-11eb-b37f-503f5601116a    1
1470a2da-c499-11eb-be3a-503f5601116a    1
14226774-c499-11eb-a8c3-503f5601116a    1
143cc99e-c499-11eb-aff2-503f5601116a    1
144b47a1-c499-11eb-b3d5-503f5601116a    1
                                       ..
14533433-c499-11eb-b60a-503f5601116a    1
1450eb64-c499-11eb-b564-503f5601116a    1
1470c9e3-c499-11eb-be48-503f5601116a    1
14409ab2-c499-11eb-b0fc-503f5601116a    1
145b23a7-c499-11eb-b845-503f5601116a    1
Name: recid, Length: 648, dtype: int64

In [30]:
matched_results.drop_duplicates(inplace=True)

In [31]:
matched_results.start_date.value_counts()

2021-06-10 18:22:00.084606+00:00    648
Name: start_date, dtype: int64

In [32]:
matched_results.to_sql(name='transactions_history',
                        con=main.engine
                        ,schema='mint'
                        ,if_exists='append'
                        ,dtype=dtypes
                        ,index=False)

In [36]:
matched_results[matched_results['recid']=='']

,trans_date,description,original_description,amount,transaction_type,category,account_name,labels,notes,recid,start_date
0,2021-04-05,BALLARD COFFEE WORK Seattle WA 04/04,SQ *BALLARD COFFEE WORK Seattle,4.13,debit,Coffee Shops,PREMIER PLUS CKG,NaN,NaN,043ca611-aab4-11eb-9943-503f5601116a,2021-05-13 01:30:29.603961+00:00
323,2021-04-05,BALLARD COFFEE WORK Seattle WA 04/04,SQ *BALLARD COFFEE WORK Seattle,4.13,debit,Coffee Shops,PREMIER PLUS CKG,NaN,NaN,043ca611-aab4-11eb-9943-503f5601116a,2021-05-13 01:30:29.603961+00:00


In [ ]:
matched_results.best_match_score.describe()